In [35]:
import pandas as pd

from elasticsearch import Elasticsearch, helpers
from elasticsearch_dsl import Search, Q
from elasticsearch.helpers import bulk

import csv

import unidecode

In [36]:
es = Elasticsearch("http://localhost:9200")

In [37]:
faq_csv = ".data/faq_with_splits_tokenized.csv"

In [38]:
df = pd.read_csv(faq_csv)

In [39]:
df.head()

,short_question,long_question,answer,main_category,sub_category,split
0,Vállsérülés gyógytorna,Vállsérülés gyógytorna Az lenne a kérdésem hog...,"edzeni kénealapjába véve mindent váll, hát, tr...",Egészség,"Sérülések, balesetek",train
1,Heti hányszor dehidtratáljam a szervezetemet c...,Heti hányszor dehidtratáljam a szervezetemet c...,"Nem értem miért akarod magad dehidratálni, nem...",Egészség,Bio életmód,train
2,A Doctor Who ban ki a vörös hajú lány,A Doctor Who ban ki a vörös hajú lány,Vagy Amy Pond vagy Donna Noble,Szórakozás,"Filmek, sorozatok",train
3,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Vettem 2 törpenyuszit a szobamba 4 5 honapja d...,Egy törpenyuszihoz nagyon sok türelem kell. Pr...,Állatok,Kisemlősök,train
4,Fityma szűkületem van,Fityma szűkületem van 15 éves fiú vagyok és mi...,Valószínű. Esetleg még a fitymaféked lehet röv...,Egészség,Férfiak egészsége,train


In [40]:
indices = es.indices.get('questions*')

for index in indices:
    es.indices.delete(index=index, ignore=[404])

C:\Users\User\anaconda3\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [41]:
def generate_data():
    
    for index, row in df.iterrows():
        yield {
            "_index": "questions_" + unidecode.unidecode(row.main_category.lower()),
            "short_question": row.short_question,
            "long_question": row.long_question,
            "category": row.main_category,
            "answer": row.answer
        }

In [42]:
bulk(es, generate_data())

(153556, [])

In [56]:
def generate_query_body(question):
    return {
        "query": {
            "match": {
                "long_question": {
                    "query": question,
                }
            }
        }
    }
    
def find_answer(query_param, generate_query, index="questions*"):
    body = generate_query_body(query_param)
    res = es.search(index=index, body=body)
    
    return res['hits']['hits'][0]['_source']

In [58]:
question = input()

result = find_answer(question, generate_query)

print(f"Short question:\n{result['short_question']}")
print()
print(f"Long question:\n{result['long_question']}")
print()
print(f"Answer:\n{result['answer']}")

Miért fáj minden hétfőn a kutyám lába?
Short question:
Miért dagadhat be a sharpei kölyök kutyám hátsó lába

Long question:
Miért dagadhat be a sharpei kölyök kutyám hátsó lába Nem hiszem h shar pei láza lenne mert nem lázas csak a bal hátsó lába fel van dagadva a bokájánál vagy csánk Nem igazán tudom hogy hívják pontosan Láthatóan fáj neki de étvágya van eszik iszik rendesen Lehet a lépcsőzés a baj

Answer:
Gondoltál màr arra, hogy orvoshoz vidd?


-------------------------------------------------------------------------------------

In [ ]:
INDEX_NAME = "questions"
INDEX_FILE = "index.json"
DATA_FILE = faq_csv
BATCH_SIZE = 1000

In [ ]:
import hu_core_ud_lg
import numpy as np

import pandas as pd

In [ ]:
data_frame = pd.read_csv(DATA_FILE)

In [ ]:
nlp = hu_core_ud_lg.load()

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('hungarian'))

def stop_word_filter(text):
    return " ". join([w for w in text if not w in stop_words])

In [ ]:
def embed_docs(docs):
    return [embed(stop_word_filter(doc)) for doc in docs]

In [ ]:
def embed(sentence):
    doc = nlp(sentence)
    lemmatized = " ".join([w.lemma_.lower() for w in doc])
    doc = nlp(lemmatized)
    helper = [doc[idx].vector for idx in range(len(doc))]
    query_vector = np.add.reduce(helper) / len(doc)
    
    return query_vector

In [ ]:
def index_data(df):
    es.indices.delete(index=INDEX_NAME, ignore=[404])

    with open(INDEX_FILE) as index_file:
        source = index_file.read().strip()
        es.indices.create(index=INDEX_NAME, body=source)
    
    docs = []
    count = 0
    
    for index, row in df.iterrows():
        docs.append(row.to_dict())
        count += 1
            
        if count % BATCH_SIZE == 0:
            index_batch(docs)
            docs = []
            print(f"\rIndexed {count} documents.", end="")

    if docs:
        index_batch(docs)
        docs = []
        print(f"\rIndexed {count} documents.", end="")

    print()
    es.indices.refresh(index=INDEX_NAME)
    print("Done indexing.")

In [ ]:
def index_batch(docs):
    short_questions = [doc["short_question"] for doc in docs]
    
    short_question_vectors = embed_docs(short_questions)

    requests = []
    
    for i, doc, in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = INDEX_NAME
        request["short_question_vector"] = short_question_vectors[i]
        requests.append(request)
    helpers.bulk(es, requests)

In [ ]:
index_data(data_frame)

In [ ]:
script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, doc['short_question_vector']) + 1.0",
            "params": {"query_vector": embed(stop_word_filter(question))}
        }
    }
}

response = es.search(
    index=INDEX_NAME,
    body={
        "size": 10,
        "query": script_query
    }
)

In [ ]:
for hit in response["hits"]["hits"]:
    print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
    print(hit["_source"]["short_question"])
    print(hit["_source"]["answer"])
    print()